## prerequirements

- (https://blog.chezo.uno/tabula-py-extract-table-from-pdf-into-python-dataframe-6c7acfa5f302 "tabula-py")

... ```sudo pip-3.6 install tabula-py # if you're using macports...```
- (https://github.com/tabulapdf/tabula-java "tabula-java")

... (https://github.com/tabulapdf/tabula-java/releases/download/v1.0.1/tabula-1.0.1-jar-with-dependencies.jar "tabula-java download")

# First we need to load the CSV metadata (column names) from the PTAX203Layout.pdf file

In [ ]:
import io, pandas
from tabula import read_pdf

PTAX203Layout = '/Users/amy/Code/chi-city/Delivery Files/Data Files/2016/PTAX203Layout.pdf'
TRED2016PTAX203 = '/Users/amy/Code/chi-city/Delivery Files/Data Files/2016/TRED2016PTAX203.txt'

In [ ]:
# How to tell tabula_py to
# . use the first row in the PDF data as pandas.df header data
# . use the first column in the PDF data as pandas.df row numbers
pdf_to_csv_to_pandas = {
    'header': 0,
    'index_col': 0
}

PTAX203_layout_df = read_pdf(PTAX203Layout, pages='all', lattice=True, pandas_options=pdf_to_csv_to_pandas)
PTAX203_layout_df

# Now we need to load the CSV using column names from the layout

In [ ]:
## This *should* work, but the data has junk that prevents it from being properly decoded :(
#
# TRED2016_df = pandas.read_csv(open(TRED2016PTAX203), names=PTAX203_layout_df['Field Name'])

### O NOES !?!
```UnicodeDecodeError: 'utf-8' codec can't decode byte 0xad in position 181978: invalid start byte``` means there is garbage in the CSV data file :(

What is ```0xad```?

```0xad``` means a binary code of ```ad``` in hexadecimal, which isn't a legal character in any UTF-8 or latin-1.

If you punch ```ad``` into your programmer's calculator app in hex mode, and then switch to decimal and octal, we can try to look up the character in some encoding tables that list various character sets to see if there's an exotic character we can replace with an ASCII candidate.

What I found was that ```0xad == 0255``` which looks suspiciously like the text was decoded from binary numbers as octal instead of decimal numbers because the strings where this appears look like multiple fields from some other database were accidentally put in a single field when uploaded into TRED2016PTAX203 data. ```255 == 0xff``` which is the ASCII NULL character, which by C language convention is a string terminator and field delimiter for raw binary data.

What we should probably do is convert those ```0xad``` characters to ```:``` or something like that. Formally, everything in the affected fields including and following ```0xad``` is probably extraneous junk that should be deleted.

In [ ]:
TRED2016PTAX203_csv_cooked = open(TRED2016PTAX203, 'rb').read().replace(bytes([0xad]),':'.encode('ascii'))

In [ ]:
TRED2016_df = pandas.read_csv(io.BytesIO(TRED2016PTAX203_csv_cooked),
                              encoding='latin_1',
                              names=PTAX203_layout_df['Field Name'])

### Note:
We still have a problem in the data where all the strings are padded with spaces and need to be stripped of any leading or trailing whitespace.

I'm working around that by setting up a Pandas boolean index *chicago_only* from mapping a lambda to check if the lowercased 'City or Village' column values start with 'chi'

In [ ]:
chicago_only = TRED2016_df['City or Village'].map(lambda city: city.lower().startswith('chi'))
dict(zip(('rows','columns'), TRED2016_df[chicago_only].shape))

In [ ]:
TRED2016_df[chicago_only]